<a href="https://colab.research.google.com/github/ab2gbl/Master2/blob/main/ITFFC/TP/ITFFC_80k_model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/liamdugan/human-detection.git

In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np

In [ ]:
!pip install opendatasets

# ROFT dataset

In [ ]:
!cp /content/drive/MyDrive/Master2/ITFFC/roft.csv ./roft.csv

In [ ]:
import pandas as pd

# Load the RoFT dataset
roft_df = pd.read_csv("roft.csv")

# Inspect the first few rows
print(roft_df.head())

# Check for missing values
print(roft_df.isnull().sum())

                               date      model  dataset  annotator group  \
0  2021-08-31 17:11:39.095000+00:00  finetuned  Recipes       1666     A   
1  2021-09-06 21:54:48.912000+00:00  finetuned  Recipes       1666     A   
2  2021-09-06 21:55:07.069000+00:00  finetuned  Recipes       1666     A   
3  2021-09-06 21:58:44.944000+00:00  finetuned  Recipes       1666     A   
4  2021-09-06 21:59:16.230000+00:00  finetuned  Recipes       1666     A   

   dec_strat_value  predicted_boundary_index  true_boundary_index  points  \
0              0.4                         0                    2       0   
1              0.4                         8                    8       5   
2              0.4                         0                    7       0   
3              0.4                         1                    7       0   
4              0.4                         1                    2       0   

                    reason  prompt  \
0  ['9123971792800820313']   13803   
1   

In [ ]:
# Drop rows where `gen_body` is NaN
roft_df = roft_df.dropna(subset=["gen_body"])

# Create a binary label column: AI-generated (1) or Human-written (0)
roft_df["label"] = roft_df["model"].apply(lambda x: 1 if x != "baseline" else 0)

# Select only the relevant columns
roft_df = roft_df[["gen_body", "label"]]

roft_df.rename(columns={'gen_body': 'text'}, inplace=True)

<ipython-input-25-bf88d409cc97>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roft_df["label"] = roft_df["model"].apply(lambda x: 1 if x != "baseline" else 0)


In [ ]:
num_rows_label_0 = roft_df[roft_df["label"] == 0].shape[0]
num_rows_label_1 = roft_df[roft_df["label"] == 1].shape[0]


print(f"Number of rows with label=0: {num_rows_label_0}")
print(f"Number of rows with label=1: {num_rows_label_1}")

Number of rows with label=0: 196
Number of rows with label=1: 24021


# Augmented data for LLM - Detect AI Generated Text

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jdragonxherrera/augmented-data-for-llm-detect-ai-generated-text")
#https://www.kaggle.com/datasets/ratthachat/writing-prompts

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: abdessamiguebli
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jdragonxherrera/augmented-data-for-llm-detect-ai-generated-text


100%|██████████| 314M/314M [00:03<00:00, 100MB/s]


In [ ]:
aug_train = pd.read_csv('/content/augmented-data-for-llm-detect-ai-generated-text/final_train.csv')
aug_test = pd.read_csv('/content/augmented-data-for-llm-detect-ai-generated-text/final_test.csv')


In [ ]:
aug_train.columns

Index(['text', 'label'], dtype='object')

In [ ]:
aug_df = pd.concat([aug_train, aug_test], ignore_index=True)

In [ ]:
aug_df.count()

,0
text,433564
label,433564


In [ ]:
aug_df['label'] = aug_df['label'].astype(int)

In [ ]:
aug_df = aug_df.dropna(how='any')

In [ ]:
aug_df.drop_duplicates(inplace=True)

In [ ]:
aug_df['label'].value_counts()

,count
label,
0,277981
1,155565


# writing prompts dataset


In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/ratthachat/writing-prompts")
# 9aba4703ac130612b1bc4c2d83ed8627
#https://www.kaggle.com/datasets/ratthachat/writing-prompts

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

In [ ]:
# Load the prompts and stories
with open("./writing-prompts/writingPrompts/train.wp_source", "r") as f:
    prompts = f.readlines()

with open("./writing-prompts/writingPrompts/train.wp_target", "r") as f:
    stories = f.readlines()

print(f"Number of prompts: {len(prompts)}")
print(f"Number of stories: {len(stories)}")

Number of prompts: 272600
Number of stories: 272600


In [ ]:
import pandas as pd

# Create a DataFrame
human_data = pd.DataFrame({
    "text": stories,  # Use stories as human-generated text
    "label": 0        # Human-generated
})

# Merge datasets

In [ ]:
# Assuming your RoFT dataset is in a DataFrame `roft_data`
df = pd.concat([roft_df, aug_df, human_data], ignore_index=True)

In [ ]:
df = aug_df.dropna(how='any')
df.drop_duplicates(inplace=True)

In [ ]:
print(df['label'].value_counts())
print("Columns in balanced_data:")
print(df.columns)

label
0    277981
1    155565
Name: count, dtype: int64
Columns in balanced_data:
Index(['text', 'label'], dtype='object')


In [ ]:
df.to_csv('AiVsHuman.csv')

In [ ]:
#!cp ./AiVsHuman.csv /content/drive/MyDrive/Master2/ITFFC/AiVsHuman.csv

# model trainning

In [ ]:
!cp /content/drive/MyDrive/Master2/ITFFC/AiVsHuman.csv ./AiVsHuman.csv

In [ ]:
import pandas as pd
df = pd.read_csv("./AiVsHuman.csv")
print(df['label'].value_counts())

label
0    277981
1    155565
Name: count, dtype: int64


In [ ]:
# Separate the dataset by label
df_label_0 = df[df['label'] == 0]
df_label_1 = df[df['label'] == 1]

# Sample 80,000 rows from each label (if there are enough rows)
df_label_1_sampled = df_label_1.sample(n=100000, random_state=42)
df_label_0_sampled = df_label_0.sample(n=100000, random_state=42)

# Concatenate the sampled data back together
df_balanced = pd.concat([df_label_0_sampled, df_label_1_sampled])

# Shuffle the resulting dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new label distribution
print(df_balanced['label'].value_counts())

label
1    100000
0    100000
Name: count, dtype: int64


In [ ]:
df = df_balanced

In [ ]:
import re

def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)

    # Remove HTML tags
    text = re.sub(r"<.*?>", '', text)

    # Remove special characters and numbers
    text = re.sub(r"[^A-Za-z\s]", '', text)

    # Remove extra whitespace
    text = re.sub(r"\s+", ' ', text).strip()

    return text


cleaned_text = clean_text("This is a test text with a URL: https://example.com and some HTML <b>bold</b>.")
print(cleaned_text)

cleaned_text = cleaned_text.lower()






This is a test text with a URL and some HTML bold


In [ ]:

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

cleaned_text = lemmatize_text(cleaned_text)
print(cleaned_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...


this is a test text with a url and some html bold


[nltk_data]   Package wordnet is already up-to-date!


In [ ]:

def preprocess_text(text):
    # Clean the text
    text = clean_text(text)

    # Lowercase
    text = text.lower()

    # Lemmatize text
    text = lemmatize_text(text)

    return text

# Apply preprocessing to the dataset
df['text'] = df['text'].apply(preprocess_text)

In [ ]:
pd.set_option('display.max_colwidth', 1000000)
df.sample().get('text').to_string()

'166372    there are many school that offer online class and distance learning many will argue that online class are much more beneficial than the average school setting but how beneficial could it possibly be for student it help those who cannot find a way to school but a majority of people who attend school are within distance so it would be pointless to force the student to take an online class they can attend on campus people often forget that there are parent and guardian to those student and online learning could possibly take a toll on the wallet of those caregiver many family already have a hard time getting by a is and adding wifi to the bill make it much more difficult distance learning is a great idea for those who live far away from campus or have a difficult time arriving to school some school already have online class so adding it into the regular curriculum would be easy there are plenty of online tool that student use to learn for instance quizlet google classroom math 

In [ ]:
'''from sklearn.utils import shuffle

# Shuffle the dataset
df = shuffle(df, random_state=42)'''


'from sklearn.utils import shuffle\n\n# Shuffle the dataset\ndf = shuffle(df, random_state=42)'

In [ ]:
from sklearn.model_selection import train_test_split

# Split into train (80%), validation (10%), and test (10%)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    test_texts, test_labels, test_size=0.5, random_state=42
)

In [ ]:
from transformers import RobertaTokenizer

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize the text
# train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
# val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)
# test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
import torch

# Custom Dataset Class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }
# Create datasets
train_dataset = CustomDataset(train_texts, train_labels , tokenizer, max_length=512)
val_dataset = CustomDataset(val_texts, val_labels , tokenizer, max_length=512)
test_dataset = CustomDataset( test_texts, test_labels , tokenizer, max_length=512)
# test_dataset = CustomDataset(test_encodings, test_labels.tolist())

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
# Load the pre-trained RoBERTa model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
# Add Early Stopping Callback
early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

# Updated Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",   # Evaluate on validation set every epoch
    save_strategy="epoch",         # Save model checkpoints
    learning_rate=3e-5,            # Fine-tuned learning rate
    per_device_train_batch_size=8, # Reduce batch size
    per_device_eval_batch_size=8,  # Match evaluation batch size
    num_train_epochs=4,            # Reduce epochs if early stopping is used
    weight_decay=0.01,             # Regularization
    logging_dir="./logs",          # Directory for logs
    fp16=True,                     # Mixed precision
    gradient_accumulation_steps=2, # Simulate larger batch size
    save_total_limit=2,            # Keep the latest 2 checkpoints
    load_best_model_at_end=True,   # Automatically load the best model
    lr_scheduler_type="linear",    # Use linear learning rate scheduling
    logging_steps=10               # Log every 10 stepWe were talking about the logistics involved in pulling off something like this, and how great it felt for him to be improvising in front of a camera again, and the curious tensions that popped up in his interview with Eminem. And then we got onto the subject of discomfort and disorientation, and the urge he has to seek out those feelings, and from there it was a quick jump to the nature of suffering. Before long we were sitting there with a plate of roast chicken and several bottles of Cholula on the table between us, both of us rubbing tears from our eyes. “The level of emotion you're getting from me right now—I'm not saying it's dishonest,” he said. “I'm just saying it's not normal. I'd really love to go to bed. I promise you, I do not spend my time on the edge of tears.”

I've easily played the recording of that conversation a dozen times, only one of them in order to transcribe. And while we spent plenty of time talking about comedy and the conventions of late-night and the sheer practical challenge of doing a show twice as long as his old one—the thing I've been thinking about the most since my time with Colbert is loss. The losses he's experienced in his life, yes, but really the meaning we all make of our losses. Deaths of loved ones, the phases of our children's lives hurtling by, jobs and relationships we never imagined would end. All of it. Among other things, our lives are compendiums of loss and change and what we make of it. I've never met anyone who's faced that reality more meaningfully than Stephen Colbert. I suppose, more than anything, that's what this story is about.

Also: ball jokes. Or the absence of them. They're doing network now, after all, and Colbert has declared a moratorium on ball jokes. (I believe I was present for the last one. It involved Greece and the Eurozone—and Paul Krugman's balls.)s
)

# Trainer with Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]  # Add early stopping callback
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Save the model
model.save_pretrained("/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model")


('/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/tokenizer_config.json',
 '/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/special_tokens_map.json',
 '/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/vocab.json',
 '/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/merges.txt',
 '/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/added_tokens.json')

In [ ]:

metrics = trainer.evaluate()
print("Validation Metrics:", metrics)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Validation Metrics: {'eval_loss': 0.012787416577339172, 'eval_accuracy': 0.998375, 'eval_precision': 0.997075310748233, 'eval_recall': 0.9997556207233627, 'eval_f1': 0.9984136668700427, 'eval_runtime': 72.3077, 'eval_samples_per_second': 110.638, 'eval_steps_per_second': 13.83, 'epoch': 4.0}


In [ ]:
from sklearn.metrics import classification_report

predictions = trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Predicted labels
predicted_labels = predictions.predictions.argmax(-1)

# Generate classification report
report = classification_report(test_labels, predicted_labels, target_names=["Human-Generated", "AI-Generated"])
print(report)


                 precision    recall  f1-score   support

Human-Generated       1.00      1.00      1.00      4008
   AI-Generated       1.00      1.00      1.00      3992

       accuracy                           1.00      8000
      macro avg       1.00      1.00      1.00      8000
   weighted avg       1.00      1.00      1.00      8000



# using the model

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model_path = "/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model"
model = RobertaForSequenceClassification.from_pretrained(model_path)

tokenizer = RobertaTokenizer.from_pretrained(model_path)

In [ ]:
import torch

In [ ]:


# Set the device (use GPU if available, else fallback to CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device
model = model.to(device)

# Input text
text = """
The Universal Declaration of Human Rights (UDHR) is a milestone document in the history of human rights. Drafted by representatives with different legal and cultural backgrounds from all regions of the world, the Declaration was proclaimed by the United Nations General Assembly in Paris on 10 December 1948 (General Assembly resolution 217 A) as a common standard of achievements for all peoples and all nations. It sets out, for the first time, fundamental human rights to be universally protected and it has been translated into over 500 languages. The UDHR is widely recognized as having inspired, and paved the way for, the adoption of more than seventy human rights treaties, applied today on a permanent basis at global and regional levels (all containing references to it in their preambles).
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move the input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Perform inference
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Get the predicted class
predicted_class = torch.argmax(logits, dim=1).item()

# Interpret the output
labels = ["Human-Generated", "AI-Generated"]
print(f"Predicted label: {labels[predicted_class]}")


Predicted label: AI-Generated


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize
# Set the device (use GPU if available, else fallback to CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device
model = model.to(device)

# Add explainability (token and sentence level)
def explain(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    # Perform inference with attention outputs
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)
        attentions = outputs.attentions  # Attention scores
        logits = outputs.logits

    # Predicted class
    predicted_class = torch.argmax(logits, dim=1).item()
    labels = ["Human-Generated", "AI-Generated"]

    # Token-level explanation
    attention_scores = attentions[0][0].mean(dim=0).mean(dim=0)  # Average over heads
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    token_attention = list(zip(tokens, attention_scores.tolist()))

    # Sentence-level explanation
    sentences = sent_tokenize(text)
    sentence_scores = []
    for sentence in sentences:
        sentence_inputs = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**sentence_inputs, output_attentions=True)
            attentions = outputs.attentions[0]
            attention_scores = attentions.mean(dim=1).mean(dim=0)
            sentence_scores.append(attention_scores.mean().item())

    return {
        "predicted_label": labels[predicted_class],
        "token_explanations": token_attention,
        "sentence_explanations": list(zip(sentences, sentence_scores))
    }


In [ ]:
test_text = """
A PC with a 550W power supply doesn't necessarily consume 550 watts every hour. The actual power consumption depends on the components inside the PC and the tasks it is performing.

If your PC uses 50% of the available power, it would consume around 275 watts per hour. Similarly, if it uses 75% of the power, it would consume around 412.5 watts per hour. The power supply's wattage rating indicates the maximum it can provide, not the actual usage.
"""
explanations = explain(test_text)

print("Predicted Label:", explanations["predicted_label"])
print("\nToken-Level Explanations:")
#for token, score in explanations["token_explanations"]:
#    print(f"{token}: {score:.3f}")

print("\nSentence-Level Explanations:")
for sentence, score in explanations["sentence_explanations"]:
    print(f"Sentence: {sentence}\nAttention Score: {score:.3f}")

Predicted Label: AI-Generated

Token-Level Explanations:

Sentence-Level Explanations:
Sentence: 
A PC with a 550W power supply doesn't necessarily consume 550 watts every hour.
Attention Score: 0.050
Sentence: The actual power consumption depends on the components inside the PC and the tasks it is performing.
Attention Score: 0.050
Sentence: If your PC uses 50% of the available power, it would consume around 275 watts per hour.
Attention Score: 0.045
Sentence: Similarly, if it uses 75% of the power, it would consume around 412.5 watts per hour.
Attention Score: 0.042
Sentence: The power supply's wattage rating indicates the maximum it can provide, not the actual usage.
Attention Score: 0.048


In [ ]:
def split_into_paragraphs(text):
    paragraphs = [p.strip() for p in text.split("\n") if p.strip()]
    return paragraphs

# Example usage
article = """
This is the first paragraph of the article. It discusses some key points.

This is the second paragraph. It delves deeper into the topic.

Finally, this is the conclusion of the article.
"""
paragraphs = split_into_paragraphs(article)
print(paragraphs)


['This is the first paragraph of the article. It discusses some key points.', 'This is the second paragraph. It delves deeper into the topic.', 'Finally, this is the conclusion of the article.']


In [ ]:
def predict_article(article, model, tokenizer):
    paragraphs = split_into_paragraphs(article)
    ai_count = 0
    human_count = 0

    for paragraph in paragraphs:
        inputs = tokenizer(paragraph, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1).item()
            if predicted_class == 1:  # AI-Generated
                ai_count += 1
            else:  # Human-Generated
                human_count += 1

    # Aggregate results
    if ai_count > human_count:
        return "AI-Generated", ai_count, human_count
    else:
        return "Human-Generated",ai_count,human_count

# Predict article
result,ai,human = predict_article(article, model, tokenizer)
print(f"The article is predicted as: {result}, ai: {ai}, human: {human}")


The article is predicted as: AI-Generated, ai: 3, human: 0


In [ ]:
def split_into_chunks(text, tokenizer, max_tokens=512):
    tokens = tokenizer.tokenize(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

# Example usage
long_paragraph = """
We were talking about the logistics involved in pulling off something like this, and how great it felt for him to be improvising in front of a camera again, and the curious tensions that popped up in his interview with Eminem. And then we got onto the subject of discomfort and disorientation, and the urge he has to seek out those feelings, and from there it was a quick jump to the nature of suffering. Before long we were sitting there with a plate of roast chicken and several bottles of Cholula on the table between us, both of us rubbing tears from our eyes. “The level of emotion you're getting from me right now—I'm not saying it's dishonest,” he said. “I'm just saying it's not normal. I'd really love to go to bed. I promise you, I do not spend my time on the edge of tears.”

I've easily played the recording of that conversation a dozen times, only one of them in order to transcribe. And while we spent plenty of time talking about comedy and the conventions of late-night and the sheer practical challenge of doing a show twice as long as his old one—the thing I've been thinking about the most since my time with Colbert is loss. The losses he's experienced in his life, yes, but really the meaning we all make of our losses. Deaths of loved ones, the phases of our children's lives hurtling by, jobs and relationships we never imagined would end. All of it. Among other things, our lives are compendiums of loss and change and what we make of it. I've never met anyone who's faced that reality more meaningfully than Stephen Colbert. I suppose, more than anything, that's what this story is about.

Also: ball jokes. Or the absence of them. They're doing network now, after all, and Colbert has declared a moratorium on ball jokes. (I believe I was present for the last one. It involved Greece and the Eurozone—and Paul Krugman's balls.)
"""
def predict_article(article, model, tokenizer):
    paragraphs = split_into_chunks(article, tokenizer)
    ai_count = 0
    human_count = 0

    for paragraph in paragraphs:
        inputs = tokenizer(paragraph, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1).item()
            if predicted_class == 1:  # AI-Generated
                ai_count += 1
            else:  # Human-Generated
                human_count += 1

    # Aggregate results
    if ai_count > human_count:
        return "AI-Generated", ai_count, human_count
    else:
        return "Human-Generated",ai_count,human_count

# Predict article
result = predict_article(long_paragraph, model, tokenizer)
print(f"The article is predicted as: {result}")


NameError: name 'model' is not defined

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Master2/ITFFC/AiVsHuman.csv")


In [ ]:
pd.set_option('display.max_colwidth', 50000)
df[df["label"]==0].sample().get('text').to_string()